# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa0734b2be0>
├── 'a' --> tensor([[ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389]])
└── 'x' --> <FastTreeValue 0x7fa0734b2ca0>
    └── 'c' --> tensor([[ 0.7225, -1.2467, -0.5805,  0.1017],
                        [-0.9994, -1.0513,  0.3705, -1.0590],
                        [-2.0516,  0.2296, -0.8679,  0.1467]])

In [4]:
t.a

tensor([[ 0.0435, -0.9193,  0.5862],
        [ 0.4862,  1.6692, -1.0389]])

In [5]:
%timeit t.a

63.8 ns ± 0.0458 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa0734b2be0>
├── 'a' --> tensor([[-0.4649,  0.1540, -0.3153],
│                   [ 0.0803,  0.2825, -1.0600]])
└── 'x' --> <FastTreeValue 0x7fa0734b2ca0>
    └── 'c' --> tensor([[ 0.7225, -1.2467, -0.5805,  0.1017],
                        [-0.9994, -1.0513,  0.3705, -1.0590],
                        [-2.0516,  0.2296, -0.8679,  0.1467]])

In [7]:
%timeit t.a = new_value

66 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389]]),
    x: Batch(
           c: tensor([[ 0.7225, -1.2467, -0.5805,  0.1017],
                      [-0.9994, -1.0513,  0.3705, -1.0590],
                      [-2.0516,  0.2296, -0.8679,  0.1467]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0435, -0.9193,  0.5862],
        [ 0.4862,  1.6692, -1.0389]])

In [11]:
%timeit b.a

60.7 ns ± 0.0287 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7040, -1.1270, -1.3569],
               [-0.3779, -2.4842, -0.6454]]),
    x: Batch(
           c: tensor([[ 0.7225, -1.2467, -0.5805,  0.1017],
                      [-0.9994, -1.0513,  0.3705, -1.0590],
                      [-2.0516,  0.2296, -0.8679,  0.1467]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.145 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 0.757 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 403 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 443 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9fb96e4f70>
├── 'a' --> tensor([[[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]],
│           
│                   [[ 0.0435, -0.9193,  0.5862],
│                    [ 0.4862,  1.6692, -1.0389]]])
└── 'x' --> <FastTreeValue 0x7f9fb96e4790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 74.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9fb96e4df0>
├── 'a' --> tensor([[ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389],
│                   [ 0.0435, -0.9193,  0.5862],
│                   [ 0.4862,  1.6692, -1.0389]])
└── 'x' --> <FastTreeValue 0x7f9fcb1b1100>
    └── 'c' --> tensor([[ 0.7225, -1.2467, -0.5805,  0.1017],
                        [-0.9994, -1.0513,  0.3705, -1.0590],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 64 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 38.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]],
       
               [[ 0.0435, -0.9193,  0.5862],
                [ 0.4862,  1.6692, -1.0389]]]),
    x: Batch(
           c: tensor([[[ 0.7225, -1.2467, -0.5805,  0.1017],
                       [-0.9994, -1.0513,  0.3705, -1.0590],
                       [-2.0516,  0.2296, -0.8679,  0.1467]],
         

In [26]:
%timeit Batch.stack(batches)

97.8 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389],
               [ 0.0435, -0.9193,  0.5862],
               [ 0.4862,  1.6692, -1.0389]]),
    x: Batch(
           c: tensor([[ 0.7225, -1.2467, -0.5805,  0.1017],
                      [-0.9994, -1.0513,  0.3705, -1.0590],
                      [-2.0516,  0.2296, -0.8679,  0.1467],
                      [ 0.7225, -1.2467, -0.5805,  0.1017],
                      [-0.9994, -

In [28]:
%timeit Batch.cat(batches)

166 µs ± 78 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

330 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
